In [1]:
import pandas as pd 
import os 
import numpy as np
import cv2

paths = []
classes = []

noTumorPath = "./mergedDataset/no_tumor"
gliomaTumorPath = "./mergedDataset/glioma_tumor"
meningiomaTumorPath = "./mergedDataset/meningioma_tumor"
pituitaryTumorPath = "./mergedDataset/pituitary_tumor"


noTumorFiles = os.listdir(noTumorPath)
gliomaFiles = os.listdir(gliomaTumorPath)
meningiomaFiles = os.listdir(meningiomaTumorPath)
pituitaryFiles = os.listdir(pituitaryTumorPath)


basePaths = [noTumorPath,gliomaTumorPath,meningiomaTumorPath,pituitaryTumorPath]
folders = [noTumorFiles,gliomaFiles,meningiomaFiles,pituitaryFiles]
for index,folder in enumerate(folders) :
    for path in folder :
        paths.append(os.path.join(basePaths[index],path))
        classes.append(index)

df = pd.DataFrame({
    "File path" : paths,
    "Class" : classes
})

df
    


,File path,Class
0,./mergedDataset/no_tumor\1.jpg,0
1,./mergedDataset/no_tumor\2.jpg,0
2,./mergedDataset/no_tumor\3.jpg,0
3,./mergedDataset/no_tumor\4.jpg,0
4,./mergedDataset/no_tumor\5.jpg,0
...,...,...
3155,./mergedDataset/pituitary_tumor\p (95).jpg,3
3156,./mergedDataset/pituitary_tumor\p (96).jpg,3
3157,./mergedDataset/pituitary_tumor\p (97).jpg,3
3158,./mergedDataset/pituitary_tumor\p (98).jpg,3


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
class imageDataset(Dataset): #creating custom data set for simplicty
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx, 0]
        label = self.dataframe.iloc[idx, 1]
        image = Image.open(img_path).convert('L')  # Convert to grayscale

        if self.transform:
            image = self.transform(image)
        
        return image, label 
    


In [4]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),     
    transforms.ToTensor(),              
])

tumorDataset = imageDataset(df, transform=transform)
tumorDataset

In [5]:

trainSize = int(0.8 * len(tumorDataset))
valSize = len(tumorDataset) - trainSize
trainDataset, validationDataset = torch.utils.data.random_split(tumorDataset, [trainSize, valSize])

trainLoader = DataLoader(trainDataset, batch_size=32, shuffle=True)
validationLoader = DataLoader(validationDataset, batch_size=32, shuffle=False)

trainDataset,validationDataset,trainLoader,validationLoader

(<torch.utils.data.dataset.Subset at 0x2b19d065d60>,
 <torch.utils.data.dataloader.DataLoader at 0x2b366388fa0>)

In [8]:
class TumorLinearNN(nn.Module):
    def __init__(self):
        super(TumorLinearNN, self).__init__()
        self.fc1 = nn.Linear(256 * 256, 1024)  # First fully connected layer
        self.fc2 = nn.Linear(1024, 512)        # Second fully connected layer
        self.fc3 = nn.Linear(512, 256)         # Third fully connected layer
        self.fc4 = nn.Linear(256, 4)           # Output layer for 4 classes

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten input
        x = torch.relu(self.fc1(x))  # First layer with ReLU activation
        x = torch.relu(self.fc2(x))  # Second layer with ReLU activation
        x = torch.relu(self.fc3(x))  # Third layer with ReLU activation
        x = self.fc4(x)              # Output layer (no activation for logits)
        return x

model = TumorLinearNN().to(device)


In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(10): 
    model.train()
    running_loss = 0.0
    for images, labels in trainLoader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(trainLoader)}')

    # Validation loop (optional)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in validationLoader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Validation Accuracy: {100 * correct / total}%')

Epoch 1, Loss: 1.2675446943391728
Validation Accuracy: 53.79746835443038%
Epoch 2, Loss: 0.9223818778991699
Validation Accuracy: 61.70886075949367%
Epoch 3, Loss: 0.7739732880381089
Validation Accuracy: 64.08227848101266%
Epoch 4, Loss: 0.6581085754346244
Validation Accuracy: 71.99367088607595%
Epoch 5, Loss: 0.6039140013199819
Validation Accuracy: 71.99367088607595%
Epoch 6, Loss: 0.5572432351263263
Validation Accuracy: 70.56962025316456%
Epoch 7, Loss: 0.48120859567123125
Validation Accuracy: 70.56962025316456%
Epoch 8, Loss: 0.3817235785949079
Validation Accuracy: 74.0506329113924%
Epoch 9, Loss: 0.3089120457349699
Validation Accuracy: 77.68987341772151%
Epoch 10, Loss: 0.24805226591946203
Validation Accuracy: 78.79746835443038%


In [10]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in trainLoader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Train Accuracy: {100 * correct / total}%')

Train Accuracy: 94.73892405063292%
